## 无标记数据的预训练

In [3]:
from importlib.metadata import version

pkgs = ["matplotlib", 
		"numpy", 
		"tiktoken", 
		"torch",
		"tensorflow" # For OpenAI's pretrained weights
		]
for p in pkgs:
	print(f"{p} version: {version(p)}")

matplotlib version: 3.7.2
numpy version: 2.0.2
tiktoken version: 0.9.0
torch version: 2.6.0+cu118
tensorflow version: 2.19.0


### 5.1 Evaluating generative text models

#### 5.1.1 GPT生成文本

In [4]:
# 未有k v cache

import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference	# 全局影响 可影响其他cell model.train()